In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

ARTICLE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""
print(summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


[{'summary_text': 'Liana Barrientos, 39, is charged with two counts of "offering a false instrument for filing in the first degree" In total, she has been married 10 times, with nine of her marriages occurring between 1999 and 2002. She is believed to still be married to four men.'}]


In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [ ]:
from transformers import pipeline, set_seed

import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"

model_ckpt = "google/pegasus-cnn_dailymail"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from datasets import load_dataset
import datasets

# Load the "samsum" dataset
dataset_dt = load_dataset("samsum")

#
#dataset_dt = load_dataset("samsum")

# Access the dataset splits (e.g., "train", "test", "validation")
train_dataset = dataset_dt["train"]
test_dataset = dataset_dt["test"]
validation_dataset = dataset_dt["validation"]

# Print some information about the dataset
print(dataset_dt)
print(train_dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})
Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 14732
})


In [ ]:
dataset_dt['train']

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 14732
})

In [ ]:
t1=dataset_dt['test'][0]['dialogue']

In [ ]:
tokenizer(t1, return_tensors="pt")

{'input_ids': tensor([[12636,   151, 10532,   108,   171,   119,   133, 17379,   131,   116,
           344,   152, 12195,   151,  1054, 55700,   553, 12636,   151,   110,
           105, 12014,   940, 56220,  2314, 12195,   151,  9752,   108,   137,
           131,   144,   258,   126,   107, 12195,   151,  5989,  9915, 12195,
           151,   285,   568,   215,   289,   166,   145,   195,   134,   109,
          1669,   424, 12636,   151,   125,   272,   131,   144,   235,   342,
           210, 12636,   151,   110,   105, 12014,   940, 56220,  2314, 12195,
           151,  1414,   131,   144,   129,  9279,   108,   178,   131,   116,
           221,   948, 12636,   151,   240,   119,   416,   167,   107,   107,
         12636,   151,   125,   131,   252,   880,   119, 50496,   342, 12195,
           151,  1205,  1352,   342,   110,   105, 12636,   151,   475, 58625,
           107,   107, 58779, 12636,   151, 43640, 12195,   151, 43640,   141,
           326,     1]]), 'attention_m

one way to generate it


In [ ]:
output=model_pegasus.generate(**tokenizer(t1, return_tensors="pt").to(device),num_beams=1, max_length=31, min_length=30, no_repeat_ngram_size=2, early_stopping=True)


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:679: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:698: UserWarning: `num_beams` is set to 1. However, `length_penalty` is set to `0.8` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `length_penalty`.
  warnings.warn(


In [ ]:
tokenizer.decode(output[0],skip_special_tokens=True)

"Hannah: I'd rather you texted him .<n>Amanda: Just text him.. Hannah file_gif> Amanda: Bye"

In [ ]:
from transformers import EvalPrediction

In [ ]:
eval=EvalPrediction(predictions=output,label_ids=tokenizer(t1, return_tensors="pt")['input_ids'])

In [ ]:
rouge_metric = load_metric('rouge')


In [ ]:
metrics=rouge_metric.compute(predictions=eval.predictions, references=eval.label_ids)

In [ ]:
metrics

{'rouge1': AggregateScore(low=Score(precision=np.float64(0.9032258064516129), recall=np.float64(0.22950819672131148), fmeasure=np.float64(0.36601307189542487)), mid=Score(precision=np.float64(0.9032258064516129), recall=np.float64(0.22950819672131148), fmeasure=np.float64(0.36601307189542487)), high=Score(precision=np.float64(0.9032258064516129), recall=np.float64(0.22950819672131148), fmeasure=np.float64(0.36601307189542487))),
 'rouge2': AggregateScore(low=Score(precision=np.float64(0.7333333333333333), recall=np.float64(0.18181818181818182), fmeasure=np.float64(0.29139072847682124)), mid=Score(precision=np.float64(0.7333333333333333), recall=np.float64(0.18181818181818182), fmeasure=np.float64(0.29139072847682124)), high=Score(precision=np.float64(0.7333333333333333), recall=np.float64(0.18181818181818182), fmeasure=np.float64(0.29139072847682124))),
 'rougeL': AggregateScore(low=Score(precision=np.float64(0.6451612903225806), recall=np.float64(0.16393442622950818), fmeasure=np.floa

In [ ]:
rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in metrics)

In [ ]:
rouge_dict

{'rouge1': np.float64(0.015181808806829837),
 'rouge2': np.float64(0.0002958456126910722),
 'rougeL': np.float64(0.015187954695590401),
 'rougeLsum': np.float64(0.015181903162681375)}

In [ ]:
pd.DataFrame(rouge_dict,index=['pegasus'])

,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.015182,0.000296,0.015188,0.015182


In [ ]:
metrics['rouge1']

AggregateScore(low=Score(precision=np.float64(0.9032258064516129), recall=np.float64(0.22950819672131148), fmeasure=np.float64(0.36601307189542487)), mid=Score(precision=np.float64(0.9032258064516129), recall=np.float64(0.22950819672131148), fmeasure=np.float64(0.36601307189542487)), high=Score(precision=np.float64(0.9032258064516129), recall=np.float64(0.22950819672131148), fmeasure=np.float64(0.36601307189542487)))

In [ ]:
def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]


In [ ]:
score = calculate_metric_on_test_ds(dataset_dt['test'], rouge_metric, model_pegasus, tokenizer, column_text = 'dialogue', column_summary='summary', batch_size=8)

100%|██████████| 103/103 [18:02<00:00, 10.51s/it]


In [ ]:
score

{'rouge1': AggregateScore(low=Score(precision=np.float64(0.008337396229931055), recall=np.float64(0.055180983608159796), fmeasure=np.float64(0.014103746417224141)), mid=Score(precision=np.float64(0.009004312071273245), recall=np.float64(0.05924661716861189), fmeasure=np.float64(0.015181808806829837)), high=Score(precision=np.float64(0.009684029204988636), recall=np.float64(0.06328115924552624), fmeasure=np.float64(0.016303144814295532))),
 'rouge2': AggregateScore(low=Score(precision=np.float64(8.097938375129254e-05), recall=np.float64(0.0005553796870769121), fmeasure=np.float64(0.000143723823548654)), mid=Score(precision=np.float64(0.00017428713594390345), recall=np.float64(0.0011820029812070036), fmeasure=np.float64(0.0002958456126910722)), high=Score(precision=np.float64(0.00028227487189828355), recall=np.float64(0.0019438852196572646), fmeasure=np.float64(0.00048058429794069224))),
 'rougeL': AggregateScore(low=Score(precision=np.float64(0.008342189260323422), recall=np.float64(0.0

In [ ]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = ['pegasus'])

,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.015182,0.000296,0.015188,0.015182
